In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
#Import package matplotlib for visualisation/plotting
# import matplotlib.pyplot as plt

# open all the yellow and green taxi file and save them into dataframe

In [2]:
raw = pd.DataFrame()
combine_raw = pd.DataFrame()
for year in range(2022, 2024):
    for month in range(1, 13):
        if (year == 2023 and month in range (5,13)):
            pass
        else:
            # Make sure the month is two digits        
            month = str(month).zfill(2)
            yellow_file_name = f'yellow_tripdata_{year}-{month}.parquet'
            print(yellow_file_name)

            try:
                raw_yellow_partition = pq.read_table(yellow_file_name)
                raw_yellow_partition = raw_yellow_partition.to_pandas()
    #             [['tpep_pickup_datetime','tpep_dropoff_datetime',
    #                                                     'passenger_count','PULocationID','DOLocationID']]
            except Exception as e:
                    print(f"Could not read file: {yellow_file_name}. Error: {e}")

            green_file_name = f'green_tripdata_{year}-{month}.parquet'
            print(green_file_name)

            try:
                raw_green_partition = pq.read_table(green_file_name)
                raw_green_partition = raw_green_partition.to_pandas()
    #             [['lpep_pickup_datetime','lpep_dropoff_datetime',
    #                                                     'passenger_count','PULocationID','DOLocationID']]

                raw_green_partition.columns = raw_green_partition.columns.str.replace('lpep_pickup_datetime', 'tpep_pickup_datetime')
                raw_green_partition.columns = raw_green_partition.columns.str.replace('lpep_dropoff_datetime', 'tpep_dropoff_datetime')
            except Exception as e:
                    print(f"Could not read file: {green_file_name}. Error: {e}")

            raw = pd.concat([raw,raw_yellow_partition, raw_green_partition],axis=0)
raw.shape

yellow_tripdata_2022-01.parquet
green_tripdata_2022-01.parquet
yellow_tripdata_2022-02.parquet
green_tripdata_2022-02.parquet
yellow_tripdata_2022-03.parquet
green_tripdata_2022-03.parquet
yellow_tripdata_2022-04.parquet
green_tripdata_2022-04.parquet
yellow_tripdata_2022-05.parquet
green_tripdata_2022-05.parquet
yellow_tripdata_2022-06.parquet
green_tripdata_2022-06.parquet
yellow_tripdata_2022-07.parquet
green_tripdata_2022-07.parquet
yellow_tripdata_2022-08.parquet
green_tripdata_2022-08.parquet
yellow_tripdata_2022-09.parquet
green_tripdata_2022-09.parquet
yellow_tripdata_2022-10.parquet
green_tripdata_2022-10.parquet
yellow_tripdata_2022-11.parquet
green_tripdata_2022-11.parquet
yellow_tripdata_2022-12.parquet
green_tripdata_2022-12.parquet
yellow_tripdata_2023-01.parquet
green_tripdata_2023-01.parquet
yellow_tripdata_2023-02.parquet
green_tripdata_2023-02.parquet
yellow_tripdata_2023-03.parquet
green_tripdata_2023-03.parquet
yellow_tripdata_2023-04.parquet
green_tripdata_2023-04.

(53439693, 22)

# clean duplicated rows

In [3]:
# duplicated rows
duplicated = raw[raw.duplicated(keep=False)]
# Show duplicate row data that can be dropped
duplicated

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,ehail_fee,trip_type,Airport_fee
3022804,2,2022-10-27 15:22:53,2022-10-27 15:23:01,1.0,0.0,1.0,N,193,193,2.0,...,0.5,0.0,0.0,0.3,3.3,0.0,0.0,NaN,NaN,NaN
3022805,2,2022-10-27 15:22:53,2022-10-27 15:23:01,1.0,0.0,1.0,N,193,193,2.0,...,0.5,0.0,0.0,0.3,3.3,0.0,0.0,NaN,NaN,NaN


In [4]:
print('Number of duplicate rows (including first) in the table is:')
duplicated.shape[0]

Number of duplicate rows (including first) in the table is:


2

No drop of duplication: High chances these 2 are different rides

# combine the pickup and dropoff data together

In [5]:
raw_pickup = raw[['tpep_pickup_datetime',
                                            'passenger_count','PULocationID']]
raw_pickup = raw_pickup.set_axis(['datetime', 'passenger_count', 'taxi_zone'], axis='columns')

raw_dropoff = raw[['tpep_dropoff_datetime',
                                            'passenger_count','DOLocationID']]
raw_dropoff = raw_dropoff.set_axis(['datetime', 'passenger_count', 'taxi_zone'], axis='columns')


combine_raw = pd.concat([raw_pickup,raw_dropoff],axis=0)
combine_raw

,datetime,passenger_count,taxi_zone
0,2022-01-01 00:35:40,2.0,142
1,2022-01-01 00:33:43,1.0,236
2,2022-01-01 00:53:21,1.0,166
3,2022-01-01 00:25:21,1.0,114
4,2022-01-01 00:36:48,1.0,68
...,...,...,...
65387,2023-04-30 22:12:00,NaN,168
65388,2023-04-30 23:43:00,NaN,170
65389,2023-04-30 23:40:00,NaN,112
65390,2023-04-30 23:57:00,NaN,236


In [6]:
 # check null value
combine_raw.isnull().sum()

datetime                 0
passenger_count    3607352
taxi_zone                0
dtype: int64

# replace the null value of passenger_count with median

In [7]:
# replace missing value with reasonable estimate -- median.
median_case_passenger_count = combine_raw['passenger_count'].median()
median_case_passenger_count
combine_raw['passenger_count'] = combine_raw['passenger_count'].replace(np.nan, median_case_passenger_count)

In [8]:
# check the missing value, make sure there is no missing value.
missing_case_passenger_count = combine_raw[combine_raw['passenger_count'].isnull()]
missing_case_passenger_count

,datetime,passenger_count,taxi_zone


# replace the "0" value of passenger_count with median

In [9]:
# check the rows of 0 value.
zero_value_case_passenger_count = combine_raw[combine_raw['passenger_count'] == 0]
zero_value_case_passenger_count

,datetime,passenger_count,taxi_zone
143,2022-01-01 00:17:34,0.0,236
144,2022-01-01 00:23:57,0.0,263
145,2022-01-01 00:31:53,0.0,140
146,2022-01-01 00:41:40,0.0,163
157,2022-01-01 00:55:36,0.0,162
...,...,...,...
60482,2023-04-30 17:20:44,0.0,65
60514,2023-04-30 18:46:26,0.0,238
60540,2023-04-30 18:49:55,0.0,97
60595,2023-04-30 18:21:29,0.0,52


In [10]:
combine_raw['passenger_count'] = combine_raw['passenger_count'].replace(0, median_case_passenger_count)

In [11]:
# check the rows of 0 value has been dropped.
zero_value_case_passenger_count = combine_raw[combine_raw['passenger_count'] == 0]
zero_value_case_passenger_count

,datetime,passenger_count,taxi_zone


In [12]:
combine_raw.dtypes

datetime           datetime64[ns]
passenger_count           float64
taxi_zone                   int64
dtype: object

In [13]:
combine_raw

,datetime,passenger_count,taxi_zone
0,2022-01-01 00:35:40,2.0,142
1,2022-01-01 00:33:43,1.0,236
2,2022-01-01 00:53:21,1.0,166
3,2022-01-01 00:25:21,1.0,114
4,2022-01-01 00:36:48,1.0,68
...,...,...,...
65387,2023-04-30 22:12:00,1.0,168
65388,2023-04-30 23:43:00,1.0,170
65389,2023-04-30 23:40:00,1.0,112
65390,2023-04-30 23:57:00,1.0,236


# group the passenger number together by the hour

In [14]:
# change the time into hour, ignore the minutes and seconds
combine_raw['datetime'] = combine_raw['datetime'].dt.floor('H')


In [15]:
df_grouped = combine_raw.groupby(['taxi_zone', 'datetime'])['passenger_count'].sum().reset_index()

In [16]:
df_grouped

,taxi_zone,datetime,passenger_count
0,1,2002-10-25 09:00:00,4.0
1,1,2002-10-27 09:00:00,3.0
2,1,2002-10-28 04:00:00,1.0
3,1,2002-10-28 05:00:00,2.0
4,1,2009-01-02 00:00:00,2.0
...,...,...,...
2032351,265,2023-04-30 20:00:00,53.0
2032352,265,2023-04-30 21:00:00,49.0
2032353,265,2023-04-30 22:00:00,68.0
2032354,265,2023-04-30 23:00:00,95.0


In [17]:
# Extract year and month
df_grouped['year_month'] = df_grouped['datetime'].dt.to_period('M')

# Extract day of week (Monday=0, Sunday=6)
# df_grouped['day'] = df_grouped['datetime'].dt.day

# Extract day of week (Monday=0, Sunday=6)
df_grouped['week'] = df_grouped['datetime'].dt.dayofweek

# Extract hour of the day
df_grouped['hour'] = df_grouped['datetime'].dt.hour

# days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
# combine_raw['week'] = combine_raw['week'].apply(lambda x: days[x])

df_grouped

,taxi_zone,datetime,passenger_count,year_month,week,hour
0,1,2002-10-25 09:00:00,4.0,2002-10,4,9
1,1,2002-10-27 09:00:00,3.0,2002-10,6,9
2,1,2002-10-28 04:00:00,1.0,2002-10,0,4
3,1,2002-10-28 05:00:00,2.0,2002-10,0,5
4,1,2009-01-02 00:00:00,2.0,2009-01,4,0
...,...,...,...,...,...,...
2032351,265,2023-04-30 20:00:00,53.0,2023-04,6,20
2032352,265,2023-04-30 21:00:00,49.0,2023-04,6,21
2032353,265,2023-04-30 22:00:00,68.0,2023-04,6,22
2032354,265,2023-04-30 23:00:00,95.0,2023-04,6,23


In [18]:
df_grouped.reset_index(drop=True).to_parquet("basic_taxi_df.parquet", index=False)

In [19]:
df = pd.read_parquet("basic_taxi_df.parquet")
df

,taxi_zone,datetime,passenger_count,year_month,week,hour
0,1,2002-10-25 09:00:00,4.0,2002-10,4,9
1,1,2002-10-27 09:00:00,3.0,2002-10,6,9
2,1,2002-10-28 04:00:00,1.0,2002-10,0,4
3,1,2002-10-28 05:00:00,2.0,2002-10,0,5
4,1,2009-01-02 00:00:00,2.0,2009-01,4,0
...,...,...,...,...,...,...
2032351,265,2023-04-30 20:00:00,53.0,2023-04,6,20
2032352,265,2023-04-30 21:00:00,49.0,2023-04,6,21
2032353,265,2023-04-30 22:00:00,68.0,2023-04,6,22
2032354,265,2023-04-30 23:00:00,95.0,2023-04,6,23


In [21]:
df_grouped.to_json("basic_taxi_df.json", orient="index", default_handler=str)

In [26]:
import json
f = open('basic_taxi_df.json')
data = json.load(f)

In [27]:
print(type(data))

<class 'dict'>


In [28]:
data

{'0': {'taxi_zone': 1,
  'datetime': 1035536400000,
  'passenger_count': 4.0,
  'year_month': '2002-10',
  'week': 4,
  'hour': 9},
 '1': {'taxi_zone': 1,
  'datetime': 1035709200000,
  'passenger_count': 3.0,
  'year_month': '2002-10',
  'week': 6,
  'hour': 9},
 '2': {'taxi_zone': 1,
  'datetime': 1035777600000,
  'passenger_count': 1.0,
  'year_month': '2002-10',
  'week': 0,
  'hour': 4},
 '3': {'taxi_zone': 1,
  'datetime': 1035781200000,
  'passenger_count': 2.0,
  'year_month': '2002-10',
  'week': 0,
  'hour': 5},
 '4': {'taxi_zone': 1,
  'datetime': 1230854400000,
  'passenger_count': 2.0,
  'year_month': '2009-01',
  'week': 4,
  'hour': 0},
 '5': {'taxi_zone': 1,
  'datetime': 1641002400000,
  'passenger_count': 2.0,
  'year_month': '2022-01',
  'week': 5,
  'hour': 2},
 '6': {'taxi_zone': 1,
  'datetime': 1641006000000,
  'passenger_count': 1.0,
  'year_month': '2022-01',
  'week': 5,
  'hour': 3},
 '7': {'taxi_zone': 1,
  'datetime': 1641009600000,
  'passenger_count': 8.0